<a href="https://colab.research.google.com/github/s1230038/autoencoder/blob/master/AutoencoderForThesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# Simple Autoencoder using Other Loss Function
# Original: https://elix-tech.github.io/ja/2016/07/17/autoencoder.html
# https://colab.research.google.com/drive/1Z_d8APkMUDwXDQIg3OI7E13vH8IZhusM?authuser=1#scrollTo=WmBfOis_mWCH
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np
from keras.datasets import mnist
from keras import backend as K
from keras import metrics
import matplotlib.pyplot as plt
from keras import losses

# imititing mean_squared_error():
# 青イルカ P.57では二乗誤差の総和を使用しているが、これをKerasで実装する場合は平均二乗誤差（mean_squared_error)
# をloss に渡す必要がある。なぜなら、Kerasはミニバッチでfit()を計算するからである。青イルカ P.27参照。
# 𝒙 ̂_𝑛 :y_pred,  𝒙_𝑛 : y_true, because x_n is training data which means label.
def i_divergence1(y_true, y_pred):
  y_true = K.clip(y_true, K.epsilon(), 1)
  y_pred = K.clip(y_pred, K.epsilon(), 1)
  return K.mean(y_pred * K.log(y_pred / y_true) - y_pred + y_true, axis=-1)

def i_divergence2(y_true, y_pred):
  y_true = K.clip(y_true, K.epsilon(), 1)
  y_pred = K.clip(y_pred, K.epsilon(), 1)
  return K.mean(y_true * K.log(y_true / y_pred) - y_true + y_pred, axis=-1)

def is_divergence1(y_true, y_pred):
  y_true = K.clip(y_true, K.epsilon(), 1)
  y_pred = K.clip(y_pred, K.epsilon(), 1)
  return K.mean( (y_pred / y_true) - K.log(y_pred / y_true) - 1, axis=-1)

def is_divergence2(y_true, y_pred):
  y_true = K.clip(y_true, K.epsilon(), 1)
  y_pred = K.clip(y_pred, K.epsilon(), 1)
  return K.mean( (y_true / y_pred) - K.log(y_true / y_pred) - 1, axis=-1)

# https://qiita.com/hiroyuki827/items/213146d551a6e2227810
def plot_history_loss(fit, name):
    # Plot the loss in the history
    axL.plot(fit.history['loss'],label="loss for training")
    axL.plot(fit.history['val_loss'],label="loss for validation")
    axL.set_title('model loss: ' + name)
    axL.set_xlabel('epoch')
    axL.set_ylabel('loss')
    axL.legend(loc='upper right')

def train(loss):
  print("start: " + loss.__name__)
  encoding_dim = 32
  input_img = Input(shape=(784,))
  #　Let us apply ReLU to both encoder activation and decoder activation。 
  encoded = Dense(encoding_dim, activation='relu')(input_img)
  decoded = Dense(784, activation='relu')(encoded) 
  autoencoder = Model(inputs=input_img, outputs=decoded) # https://medium.com/@zhuixiyou/userwarning-update-your-model-call-to-the-keras-2-api-8a6a5955daac
  # autoencoderでは、教師データにラベルを使わないためaccuracyの計算は不要。
  autoencoder.compile(optimizer='adam', loss=loss)

  fit = autoencoder.fit(x_train, x_train,
                  # nb_epoch=50,
                  nb_epoch=5,
                  batch_size=256,
                  shuffle=True,
                  validation_data=(x_test, x_test))

  # グラフの表示と保存
  plot_history_loss(fit, loss.__name__)
  fig.savefig( loss.__name__ + '_loss_history.png' )
  plt.close()
  
  # 学習した重みを保存
  autoencoder.save_weights( loss.__name__ + 'autoencoder.h5')


fig, (axL, axR) = plt.subplots(ncols=2, figsize=(10,4)) # グラフの表示準備
# Download MNIST and standardize, learning
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# we will restrict domain of definition of the input data to the below expression with normalization of the input.
x_train = x_train.astype('float32') / 255. # 画像データは0から1の実数値を取るように規格化
x_test = x_test.astype('float32') / 255.   # {0,1}の二値ではなく実数値であることに注意
# x_trainは (60000, 28, 28) という形をしていますが、784次元の入力になるように (60000, 784) に変形
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
# loss functions
lossfs = [i_divergence1, i_divergence2, is_divergence1, is_divergence2, losses.mean_squared_error]
for loss in lossfs:
  train(loss)
  

start: i_divergence1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 33us/step - loss: 0.1363 - val_loss: 0.1325
Epoch 2/5
60000/60000 [==============================] - 1s 22us/step - loss: 0.1306 - val_loss: 0.1324
Epoch 3/5
60000/60000 [==============================] - 1s 22us/step - loss: 0.1306 - val_loss: 0.1324
Epoch 4/5
60000/60000 [==============================] - 1s 22us/step - loss: 0.1306 - val_loss: 0.1324
Epoch 5/5
60000/60000 [==============================] - 1s 22us/step - loss: 0.1294 - val_loss: 0.1290
start: i_divergence2
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 32us/step - loss: 0.2245 - val_loss: 0.1422
Epoch 2/5
60000/60000 [==============================] - 1s 22us/step - loss: 0.1388 - val_loss: 0.1308
Epoch 3/5
60000/60000 [==============================] - 1s 22us/step - loss: 0.1275 - val_loss: 0.1201
Epoch 4/5
60000/60000 [========================

In [47]:
import matplotlib.pyplot as plt

for loss in lossfs:
  print("show: " + loss.__name__)
  # 保存した重みを読み込み
  autoencoder.load_weights(loss.__name__ + 'autoencoder.h5')
  decoded_imgs = autoencoder.predict(x_test)
  # 0-9を表示する
  plt.figure(figsize=(20, 4))
  label = 0
  for i in range(1000):
      if label > 10:
          break
      elif label != y_test[i]:
          continue
      # オリジナルのテスト画像を表示
      ax = plt.subplot(2, 10, label+1)
      plt.imshow(x_test[i].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

      # 変換された画像を表示
      ax = plt.subplot(2, 10, label+1+10)
      plt.imshow(decoded_imgs[i].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      
      label+=1
  plt.show()


show: i_divergence1


TypeError: ignored

In [45]:
# 保存した重みのダウンロード
from google.colab import files
# !zip -r autoencoders.zip *autoencoder.h5
!zip -r loss_history.zip *_loss_history.png
# files.download('autoencoders.zip')
files.download('loss_history.zip')

updating: i_divergence1_loss_history.png (deflated 6%)
updating: i_divergence2_loss_history.png (deflated 7%)
updating: is_divergence1_loss_history.png (deflated 9%)
updating: is_divergence2_loss_history.png (deflated 12%)
updating: mean_squared_error_loss_history.png (deflated 10%)
